In [7]:
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.sql.types import *

from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline
import sparknlp
from pyspark.sql import SparkSession


from sparknlp.annotator import *
from pyspark.ml import Pipeline
#sc = SparkContext()
import sparknlp
spark = sparknlp.start()

/home/yogender/.local/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [8]:
sqlContext = SQLContext(spark)

/home/yogender/.local/lib/python3.6/site-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [9]:
rdd_l = spark.sparkContext.wholeTextFiles("./*.html")


In [10]:
df = rdd_l.toDF(schema=["filename", "text"]).select("text")

In [11]:
df.show(10)

+--------------------+
|                text|
+--------------------+
|<!DOCTYPE html><h...|
|<!DOCTYPE html><h...|
|<!DOCTYPE html><h...|
|<!DOCTYPE html><h...|
|<!DOCTYPE html><h...|
|<!DOCTYPE html><h...|
|<!DOCTYPE html><h...|
|<!DOCTYPE html><h...|
|<!DOCTYPE html><h...|
|<!DOCTYPE html><h...|
+--------------------+



In [21]:
rules = '''

\d+&\w+;\d+&\w+;Kč* 

'''
#\S*\d+\S* Kč* 
with open('regex_rules.txt', 'w') as f:
    
    f.write(rules)

In [6]:
RegexMatcher().extractParamMap()

NameError: name 'RegexMatcher' is not defined

In [23]:


#match_df.select('regex_matches.result').take(3)

In [24]:




documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("assembled")

regex_matcher = RegexMatcher()\
    .setInputCols('assembled')\
    .setStrategy("MATCH_ALL")\
    .setOutputCol("regex_matches")\
    .setExternalRules(path='./regex_rules.txt', delimiter=',')

nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    regex_matcher
 ])

pipelineModel = nlpPipeline.fit(df)
#df = pipelineModel.transform(df)
#empty_df = spark.createDataFrame([['']]).toDF("text")

#pipelineModel = nlpPipeline.fit(empty_df)

match_df = pipelineModel.transform(df)

match_df.select('regex_matches.result').show(truncate=50)

+----------------------+
|                result|
+----------------------+
|[500&nbsp;000&nbsp;Kč]|
|[200&nbsp;000&nbsp;Kč]|
|[854&nbsp;900&nbsp;Kč]|
|[565&nbsp;000&nbsp;Kč]|
|[475&nbsp;000&nbsp;Kč]|
|[500&nbsp;000&nbsp;Kč]|
|[530&nbsp;000&nbsp;Kč]|
|[511&nbsp;000&nbsp;Kč]|
|[850&nbsp;000&nbsp;Kč]|
|[490&nbsp;000&nbsp;Kč]|
+----------------------+



In [25]:
match_df.select('regex_matches.result').show(truncate=100)

+----------------------+
|                result|
+----------------------+
|[500&nbsp;000&nbsp;Kč]|
|[200&nbsp;000&nbsp;Kč]|
|[854&nbsp;900&nbsp;Kč]|
|[565&nbsp;000&nbsp;Kč]|
|[475&nbsp;000&nbsp;Kč]|
|[500&nbsp;000&nbsp;Kč]|
|[530&nbsp;000&nbsp;Kč]|
|[511&nbsp;000&nbsp;Kč]|
|[850&nbsp;000&nbsp;Kč]|
|[490&nbsp;000&nbsp;Kč]|
+----------------------+



In [26]:
match_df.select('text', 'regex_matches.result')\
    .toDF('text', 'regex_matches.result')\
    .show(truncate=50)

+--------------------------------------------------+----------------------+
|                                              text|  regex_matches.result|
+--------------------------------------------------+----------------------+
|<!DOCTYPE html><html lang="cs" ng-app="sreality...|[500&nbsp;000&nbsp;Kč]|
|<!DOCTYPE html><html lang="cs" ng-app="sreality...|[200&nbsp;000&nbsp;Kč]|
|<!DOCTYPE html><html lang="cs" ng-app="sreality...|[854&nbsp;900&nbsp;Kč]|
|<!DOCTYPE html><html lang="cs" ng-app="sreality...|[565&nbsp;000&nbsp;Kč]|
|<!DOCTYPE html><html lang="cs" ng-app="sreality...|[475&nbsp;000&nbsp;Kč]|
|<!DOCTYPE html><html lang="cs" ng-app="sreality...|[500&nbsp;000&nbsp;Kč]|
|<!DOCTYPE html><html lang="cs" ng-app="sreality...|[530&nbsp;000&nbsp;Kč]|
|<!DOCTYPE html><html lang="cs" ng-app="sreality...|[511&nbsp;000&nbsp;Kč]|
|<!DOCTYPE html><html lang="cs" ng-app="sreality...|[850&nbsp;000&nbsp;Kč]|
|<!DOCTYPE html><html lang="cs" ng-app="sreality...|[490&nbsp;000&nbsp;Kč]|
+-----------

In [397]:
rules2 = '''

\w*tion\-\w*ext\s+\w*ng\-\w*binding\"\>\w*.........................


'''
#\w*tion\-\w*ext\s+\w*ng\-\w*binding\"\>\w*
#/<title>([^<]+)<\/title>/
#\w*tion\-\w*ext\s+\w*ng\-\w*binding\"\>\w+|\s.$\<\/
#\S*\d+\S* Kč*
#\d+&\w+;\d+&\w+;Kč* 
with open('regex_rules2.txt', 'w') as f:
    
    f.write(rules2)

In [398]:
RegexMatcher().extractParamMap()

documentAssembler2 = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("assembled")

regex_matcher2 = RegexMatcher()\
    .setInputCols('assembled')\
    .setStrategy("MATCH_ALL")\
    .setOutputCol("regex_address")\
    .setExternalRules(path='./regex_rules2.txt', delimiter=',')

nlpPipeline2 = Pipeline(stages=[
    documentAssembler2, 
    regex_matcher2
 ])

In [413]:

pipelineModel2 = nlpPipeline2.fit(df)

match_df2 = pipelineModel2.transform(df)

match_df2.select('regex_address.result').show(truncate=400)

+------------------------------------------------------------+
|                                                      result|
+------------------------------------------------------------+
|   [location-text ng-binding">Masná, Praha 1 - Staré Město<]|
|    [location-text ng-binding">Velvarská, Horoměřice</span>]|
|   [location-text ng-binding">Libušina, Karlovy Vary</span>]|
|    [location-text ng-binding">Pod Lipami, Praha 3 - Žižkov]|
|[location-text ng-binding">Drnovská, Praha 6 - Ruzyně</span]|
|      [location-text ng-binding">V Průhoně, Mukařov - Žerno]|
| [location-text ng-binding">Mladoboleslavská, Mělník</span>]|
|[location-text ng-binding">Grafická, Praha 5 - Smíchov</spa]|
|[location-text ng-binding">Sametová, Liberec - Liberec VI-R]|
|[location-text ng-binding">Jakubská, Praha 1 - Staré Město<]|
+------------------------------------------------------------+



In [414]:
match_df2.printSchema()

root
 |-- text: string (nullable = true)
 |-- assembled: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- regex_address: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull 

In [416]:
from pyspark.sql import functions as F

match_df2.select(F.explode('regex_address.result')).show(truncate=False)

+----------------------------------------------------------+
|col                                                       |
+----------------------------------------------------------+
|location-text ng-binding">Masná, Praha 1 - Staré Město<   |
|location-text ng-binding">Velvarská, Horoměřice</span>    |
|location-text ng-binding">Libušina, Karlovy Vary</span>   |
|location-text ng-binding">Pod Lipami, Praha 3 - Žižkov    |
|location-text ng-binding">Drnovská, Praha 6 - Ruzyně</span|
|location-text ng-binding">V Průhoně, Mukařov - Žerno      |
|location-text ng-binding">Mladoboleslavská, Mělník</span> |
|location-text ng-binding">Grafická, Praha 5 - Smíchov</spa|
|location-text ng-binding">Sametová, Liberec - Liberec VI-R|
|location-text ng-binding">Jakubská, Praha 1 - Staré Město<|
+----------------------------------------------------------+



In [417]:
result_df2 = match_df2.select(F.explode('regex_address.result').alias('location-text ng-binding')).toPandas()
result_df2

,location-text ng-binding
0,"location-text ng-binding"">Masná, Praha 1 - Sta..."
1,"location-text ng-binding"">Velvarská, Horoměřic..."
2,"location-text ng-binding"">Libušina, Karlovy Va..."
3,"location-text ng-binding"">Pod Lipami, Praha 3 ..."
4,"location-text ng-binding"">Drnovská, Praha 6 - ..."
5,"location-text ng-binding"">V Průhoně, Mukařov -..."
6,"location-text ng-binding"">Mladoboleslavská, Mě..."
7,"location-text ng-binding"">Grafická, Praha 5 - ..."
8,"location-text ng-binding"">Sametová, Liberec - ..."
9,"location-text ng-binding"">Jakubská, Praha 1 - ..."


NameError: name 'RegexMatcher' is not defined